In [ ]:
import sympy as sp
import numpy as np
from sympy import latex

import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from modules.ur3_symbols import *
from modules.kinematics import get_forward_kinematics, get_transform

In [ ]:
# this is the official dh table from universal robotics, not the one derived by Coppelia's frame assignments

ur3_dh_table = [
    {'theta':q_syms[0], 'a':0, 'd':d_syms[0], 'alpha':sp.pi/2},
    {'theta':q_syms[1], 'a':a_syms[1], 'd':0, 'alpha':0},
    {'theta':q_syms[2], 'a':a_syms[2], 'd':0, 'alpha':0},
    {'theta':q_syms[3], 'a':0, 'd':d_syms[3], 'alpha':sp.pi/2},
    {'theta':q_syms[4], 'a':0, 'd':d_syms[4], 'alpha':-sp.pi/2},
    {'theta':q_syms[5], 'a':0, 'd':d_syms[5], 'alpha':0},
]

ur3_subs = {
    d_syms[0]: 0.1519,
    a_syms[1]: -0.24365,
    a_syms[2]: -0.21325,
    d_syms[3]: 0.11235,
    d_syms[4]: 0.08535,
    d_syms[5]: 0.0819
}

n = len(ur3_dh_table)

In [ ]:
ur3_transforms = get_forward_kinematics(ur3_dh_table)
ur3_transforms = [sp.trigsimp(transform) for transform in ur3_transforms]

In [ ]:
# Extract the necessary components for the jacobian
origins = [ H[:3, -1] for H in ur3_transforms ]
z_vectors = [ H[:3, 2] for H in ur3_transforms[1:]]
center_of_masses = [r_c1, r_c2, r_c3, r_c4, r_c5, r_c6]

In [ ]:
def build_symbolic_com_jacobians(origins, z_vectors, com_positions):
    """
    Constructs the Jacobian matrices (linear + angular) for the centers of mass of each link.

    Parameters:
    - origins: list of sympy Matrix, position of each joint frame (length N+1)
    - z_vectors: list of sympy Matrix, z-axis of each joint frame (length N)
    - com_positions: list of sympy Matrix, center of mass positions for each link (length N)

    Returns:
    - jacobians: list of sympy Matrix, each 6xN Jacobian for a link's CoM
    """
    num_joints = len(z_vectors)
    jacobians = []

    for link_idx in range(num_joints):
        J = sp.zeros(3, num_joints)
        o_c = com_positions[link_idx]
        for j in range(link_idx + 1):
            o_n = origins[j]
            z_n = z_vectors[j]
            J[:3, j] = sp.simplify(z_n.cross(o_c - o_n) ) # Linear velocity Jacobian
            # J[3:, j] = z_n                   # Angular velocity Jacobian
        jacobians.append(J)

    return jacobians



In [ ]:
jacobians = build_symbolic_com_jacobians(origins, z_vectors, center_of_masses)
jacobians[0]

In [ ]:
def compute_inertia_matrices(jacobians, masses):
    inertia_matrices = []

    for J, m in zip(jacobians, masses):
        D = m * J.T * J
        # D = sp.simplify(D)
        inertia_matrices.append(D)
    
    return inertia_matrices

In [ ]:
inertia_matrices = compute_inertia_matrices(jacobians[4:], m_syms[4:])

In [ ]:
# sum all of the inertia matrices for each link
full_inertia_mat = sum(inertia_matrices, sp.zeros(*inertia_matrices[0].shape))

In [ ]:
def compute_coriolis_matrix(D, q_syms:list, dq_syms:list):
    '''
    Args:
        D: Inertia Matrix
        q_syms: list of symbolic joint variables
        dq_syms: list of symbolic time derivative of joint variables
    '''
    coriolis = sp.Matrix.zeros(n)

    for k in range(n):
        for j in range(n):
            c_kj = 0
            for i in range(n):
                c_ijk = (sp.diff(D[k, j], q_syms[j]) 
                         + sp.diff(D[k, i], q_syms[j])
                         - sp.diff(D[i, j], q_syms[k]))/2
                c_kj += c_ijk * dq_syms[i]
            # c_kj = sp.simplify(c_kj)
            # takes too long to simplify
            coriolis[k, j] = c_kj
    
    return coriolis
                           

In [ ]:
# do not try to print the full matrix
coriolis = compute_coriolis_matrix(full_inertia_mat, q_syms, dq_syms)
